In [ ]:
%matplotlib inline

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import csv
import os
import numpy
import time
import statistics

In [ ]:
#the following codes reads the csv data files and converts them to trainloader and testloader

original_directory = os.getcwd()
TrainList = []
LabelList = []

directory = os.path.join(r'Data_Point\Training\Fault',"23")
for root,dirs,files in os.walk(directory):
    for file in files:
        if file.endswith(".csv"):
            os.chdir(directory)
            with open(file, 'r') as f:
                reader = csv.reader(f,quoting=csv.QUOTE_NONNUMERIC)
                your_list = list(reader)
                if (len(your_list) == 23 and len(your_list[0]) == 23):
                    #your_list = normalise_list(your_list,10)
                    TrainList.append(your_list)
                    LabelList.append(1)
            os.chdir(original_directory)
            
directory = os.path.join(r'Data_Point\Training\Non-Fault',"23")
for root,dirs,files in os.walk(directory):
    for file in files:
        if file.endswith(".csv"):
            os.chdir(directory)
            with open(file, 'r') as f:
                reader = csv.reader(f,quoting=csv.QUOTE_NONNUMERIC)
                your_list = list(reader)
                if (len(your_list) == 23 and len(your_list[0]) == 23):
                    #your_list = normalise_list(your_list,10)
                    TrainList.append(your_list)
                    LabelList.append(0)
            os.chdir(original_directory)

#print(TrainList[0])
inputs = torch.FloatTensor(TrainList)
inputs = inputs.reshape(inputs.shape[0],1,23,23)
#labellist = [0,0,1,1]
labels = torch.tensor(LabelList, dtype=torch.long)
train_data = []
for i in range(len(inputs)):
#    print(csvlabels[i])
    train_data.append([inputs[i], labels[i]])

trainloader = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=10)
print('the size of training dataset and its labels')
print(inputs.shape);
print(labels.shape);

TestList = []
OLabelList = []

directory = os.path.join(r'Data_Point\Testing\Fault',"23")
for root,dirs,files in os.walk(directory):
    for file in files:
        if file.endswith(".csv"):
            os.chdir(directory)
            with open(file, 'r') as f:
                reader = csv.reader(f,quoting=csv.QUOTE_NONNUMERIC)
                your_list = list(reader)
                if (len(your_list) == 23 and len(your_list[0]) == 23):
                    #your_list = normalise_list(your_list,10)
                    TestList.append(your_list)
                    OLabelList.append(1)
            os.chdir(original_directory)
            
directory = os.path.join(r'Data_Point\Testing\Non-Fault',"23")
for root,dirs,files in os.walk(directory):
    for file in files:
        if file.endswith(".csv"):
            os.chdir(directory)
            with open(file, 'r') as f:
                reader = csv.reader(f,quoting=csv.QUOTE_NONNUMERIC)
                your_list = list(reader)
                if (len(your_list) == 23 and len(your_list[0]) == 23):
                    #your_list = normalise_list(your_list,10)
                    TestList.append(your_list)
                    OLabelList.append(0)
            os.chdir(original_directory)

#print(TrainList[0])
test_inputs = torch.FloatTensor(TestList)
test_inputs = test_inputs.reshape(test_inputs.shape[0],1,23,23)
#labellist = [0,0,1,1]
test_labels = torch.tensor(OLabelList, dtype=torch.long)

test_data = []
for i in range(len(test_inputs)):
#    print(csvlabels[i])
    test_data.append([test_inputs[i], test_labels[i]])
    
testloader = torch.utils.data.DataLoader(test_data, shuffle=True, batch_size=10)
print('the size of testing dataset and its labels')
print(test_inputs.shape);
print(test_labels.shape);

In [ ]:
# define the two calsses of data for the labels 0 and 1 respectively 

classes = ('non_fault', 'fault')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# functions to show an seismic data
print('three random examples of training fault data')
plt.subplot(1,3,1)
plt.imshow(inputs[1200].reshape(23,23), cmap=plt.cm.seismic)
plt.subplot(1,3,2)
plt.imshow(inputs[1600].reshape(23,23), cmap=plt.cm.seismic)
plt.subplot(1,3,3)
plt.imshow(inputs[8400].reshape(23,23), cmap=plt.cm.seismic)

In [ ]:
print('three random examples of training non-fault data')
plt.subplot(1,3,1)
plt.imshow(inputs[10800].reshape(23,23), cmap=plt.cm.seismic)
plt.subplot(1,3,2)
plt.imshow(inputs[14300].reshape(23,23), cmap=plt.cm.seismic)
plt.subplot(1,3,3)
plt.imshow(inputs[17700].reshape(23,23), cmap=plt.cm.seismic)

In [ ]:
print('four random examples of testing data')
plt.subplot(1,4,1)
plt.imshow(test_inputs[450].reshape(23,23), cmap=plt.cm.seismic)
plt.subplot(1,4,2)
plt.imshow(test_inputs[2300].reshape(23,23), cmap=plt.cm.seismic)
plt.subplot(1,4,3)
plt.imshow(test_inputs[4600].reshape(23,23), cmap=plt.cm.seismic)
plt.subplot(1,4,4)
plt.imshow(test_inputs[5800].reshape(23,23), cmap=plt.cm.seismic)

In [ ]:
#define CNN

import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 3, 1, 1)
        self.conv2 = nn.Conv2d(6, 16, 3, 1, 1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(16, 25, 3, 1, 1)
        self.conv4 = nn.Conv2d(25, 32, 3, 1, 1)
        self.fc1 = nn.Linear(800, 2048)
        self.fc2 = nn.Linear(2048, 512)
        self.fc3 = nn.Linear(512, 32)
        self.fc4 = nn.Linear(32, 2)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        x = self.pool(F.relu(self.conv4(x)))
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        x = F.softmax(x, dim=1)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()


In [ ]:
#define loss function and optimizer

import torch.optim as optim

criterion = nn.CrossEntropyLoss()
#optimizer = optim.Adam(net.parameters(), lr=0.0001)
#optimizer = optim.SGD(net.parameters(), lr=0.0001, momentum=0.9)
#optimizer = optim.Adadelta(net.parameters(), lr=0.0001, rho=0.9, eps=1e-06, weight_decay=0)
optimizer = optim.Adagrad(net.parameters(), lr=0.0001, lr_decay=0, weight_decay=0, initial_accumulator_value=0, eps=1e-10)
#optimizer = optim.AdamW(net.parameters(), lr=0.0001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.01, amsgrad=False)
#optimizer = optim.ASGD(net.parameters(), lr=0.0001, lambd=0.0001, alpha=0.75, t0=1000000.0, weight_decay=0)
#optimizer = optim.RMSprop(net.parameters(), lr=0.0001, alpha=0.99, eps=1e-08, weight_decay=0, momentum=0, centered=False)
#optimizer = optim.SGD(net.parameters(), lr=0.0001, momentum=0, dampening=0, weight_decay=0, nesterov=False)
#optimizer = optim.Rprop(net.parameters(), lr=0.0001, etas=(0.5, 1.2), step_sizes=(1e-06, 50))

In [ ]:
#train the network for 20 epochs

for epoch in range(20): # loop over the dataset multiple times

    running_loss = 0.0
    t0 = time.time()
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 600 == 599:    # print every 600 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 600))
            running_loss = 0.0
    print('{} seconds'.format(time.time() - t0))

print('Finished Training')

In [ ]:
#save the network parameters

PATH = './params/seismic_net_23.pth'
torch.save(net.state_dict(), PATH)

In [ ]:
#test sample data from testing

dataiter = iter(testloader)
test_ips, labels = dataiter.next()
net = Net()
net.load_state_dict(torch.load(PATH))
outputs = net(test_ips)
print('predicted non fault and fault probabilities')
print(outputs)
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                              for j in range(1)))

In [ ]:
#test the performance of the network on whole dataset

correct = 0.00
total = 0.00
with torch.no_grad():
    for data in testloader:
        seis_data, labels = data
        outputs = net(seis_data)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test dataset: %d %%' % (
    100 * correct / total))
print(total)

In [ ]:
#this part runs the code on GPU 10 times and reports the average accuracy

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Assuming that we are on a CUDA machine, this should print a CUDA device:
print(device)

acc_list = []
for j in range(10):
    class Net(nn.Module):
        def __init__(self):
            super(Net, self).__init__()
            self.conv1 = nn.Conv2d(1, 6, 3, 1, 1)
            self.conv2 = nn.Conv2d(6, 16, 3, 1, 1)
            self.pool = nn.MaxPool2d(2, 2)
            self.conv3 = nn.Conv2d(16, 25, 3, 1, 1)
            self.conv4 = nn.Conv2d(25, 32, 3, 1, 1)
            self.fc1 = nn.Linear(800, 2048)
            self.fc2 = nn.Linear(2048, 512)
            self.fc3 = nn.Linear(512, 32)
            self.fc4 = nn.Linear(32, 2)

        def forward(self, x):
            x = F.relu(self.conv1(x))
            x = self.pool(F.relu(self.conv2(x)))
            x = F.relu(self.conv3(x))
            x = self.pool(F.relu(self.conv4(x)))
            x = x.view(-1, self.num_flat_features(x))
            x = F.relu(self.fc1(x))
            x = F.relu(self.fc2(x))
            x = F.relu(self.fc3(x))
            x = self.fc4(x)
            x = F.softmax(x, dim=1)
            return x
    
        def num_flat_features(self, x):
            size = x.size()[1:]  # all dimensions except the batch dimension
            num_features = 1
            for s in size:
                num_features *= s
            return num_features

    net = Net()
    net.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adagrad(net.parameters(), lr=0.0001, lr_decay=0, weight_decay=0, initial_accumulator_value=0, eps=1e-10)
    
    for epoch in range(20): # loop over the dataset multiple times
        running_loss = 0.0
        t0 = time.time()
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data[0].to(device), data[1].to(device)
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

    print('Finished Training for %d'% (j+1))
    
    PATH = './seismic_net_23.pth'
    torch.save(net.state_dict(), PATH)
    dataiter = iter(testloader)
    test_ips, labels = dataiter.next()[0].to(device), dataiter.next()[1].to(device)
    net = Net()
    net.load_state_dict(torch.load(PATH))
    net.to(device)
    
    correct = 0.00
    total = 0.00
    with torch.no_grad():
        for data in testloader:
            seis_data, labels = data[0].to(device), data[1].to(device)
            outputs = net(seis_data)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accur = 100*(correct/total)
    acc_list.append(accur)
avg_acc = statistics.mean(acc_list)
print ("the avg acc is ")
print(avg_acc)

In [ ]:
#this part is same code as above (runs 10 times and reports avg accuracy) but doesn't need GPU to run

acc_list = []
for j in range(10):
    class Net(nn.Module):
        def __init__(self):
            super(Net, self).__init__()
            self.conv1 = nn.Conv2d(1, 6, 3, 1, 1)
            self.conv2 = nn.Conv2d(6, 16, 3, 1, 1)
            self.pool = nn.MaxPool2d(2, 2)
            self.conv3 = nn.Conv2d(16, 25, 3, 1, 1)
            self.conv4 = nn.Conv2d(25, 32, 3, 1, 1)
            self.fc1 = nn.Linear(800, 2048)
            self.fc2 = nn.Linear(2048, 512)
            self.fc3 = nn.Linear(512, 32)
            self.fc4 = nn.Linear(32, 2)

        def forward(self, x):
            x = F.relu(self.conv1(x))
            x = self.pool(F.relu(self.conv2(x)))
            x = F.relu(self.conv3(x))
            x = self.pool(F.relu(self.conv4(x)))
            x = x.view(-1, self.num_flat_features(x))
            x = F.relu(self.fc1(x))
            x = F.relu(self.fc2(x))
            x = F.relu(self.fc3(x))
            x = self.fc4(x)
            x = F.softmax(x, dim=1)
            return x
    
        def num_flat_features(self, x):
            size = x.size()[1:]  # all dimensions except the batch dimension
            num_features = 1
            for s in size:
                num_features *= s
            return num_features

    net = Net()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adagrad(net.parameters(), lr=0.0001, lr_decay=0, weight_decay=0, initial_accumulator_value=0, eps=1e-10)
    
    for epoch in range(20): # loop over the dataset multiple times
        running_loss = 0.0
        t0 = time.time()
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

    print('Finished Training for %d'% (j+1))
    
    PATH = './seismic_net_23.pth'
    torch.save(net.state_dict(), PATH)
    dataiter = iter(testloader)
    test_ips, labels = dataiter.next()
    net = Net()
    net.load_state_dict(torch.load(PATH))
    
    correct = 0.00
    total = 0.00
    with torch.no_grad():
        for data in testloader:
            seis_data, labels = data
            outputs = net(seis_data)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accur = 100*(correct/total)
    acc_list.append(accur)
avg_acc = statistics.mean(acc_list)
print ("the avg acc is ")
print(avg_acc)